In [ ]:
%pip install selenium
%pip install streamlit
%pip install mysql

In [1]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.Collecting mysql-connector-python
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta

ERROR: Exception:
Traceback (most recent call last):
  File "c:\all about python\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\all about python\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "c:\all about python\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\all about python\.venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Siddharth\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "c:\Users\Siddharth\AppData\Local\Programs\Python\Python312\Lib\socket.py

ModuleNotFoundError: No module named 'mysql'

In [2]:

from mysql import connector
connection = connector.connect(
    host = "localhost", 
    user= "root",
    password= "123456789"

)
cursor= connection.cursor()
cursor
query = "Create database Redbus_data"
cursor.execute(query)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import mysql.connector
import pandas as pd

# Set up Selenium WebDriver
service = Service('path_to_chromedriver')  # Provide the correct path to chromedriver
driver = webdriver.Chrome(service=service)

# Open the Redbus website
driver.get("https://www.redbus.in/")
time.sleep(5)

# Example: Searching for buses between two locations
source = driver.find_element(By.ID, "src")
destination = driver.find_element(By.ID, "dest")

source.send_keys("Chennai")
destination.send_keys("Bangalore")

date = driver.find_element(By.ID, "onward_cal")
date.click()
# Choose a date logic can be added here

search_button = driver.find_element(By.ID, "search_btn")
search_button.click()
time.sleep(10)

# Scrape bus data
buses = driver.find_elements(By.CLASS_NAME, "bus-item")
data = []
for bus in buses:
    try:
        name = bus.find_element(By.CLASS_NAME, "travels").text
        bustype = bus.find_element(By.CLASS_NAME, "bus-type").text
        departure = bus.find_element(By.CLASS_NAME, "dp-time").text
        duration = bus.find_element(By.CLASS_NAME, "dur").text
        arrival = bus.find_element(By.CLASS_NAME, "bp-time").text
        rating = bus.find_element(By.CLASS_NAME, "rating").text
        price = bus.find_element(By.CLASS_NAME, "seat-fare span").text
        availability = bus.find_element(By.CLASS_NAME, "seat-left").text

        data.append({
            'name': name,
            'bustype': bustype,
            'departure': departure,
            'duration': duration,
            'arrival': arrival,
            'rating': rating,
            'price': price,
            'availability': availability
        })
    except Exception as e:
        print("Error:", e)

# Close the driver
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV for verification
df.to_csv('redbus_data.csv', index=False)


ModuleNotFoundError: No module named 'mysql'

SQL Script for database creation and insertion.

In [ ]:
# Database connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="redbus_data"
)
cursor = connection.cursor()

# Create the table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_routes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        bustype VARCHAR(255),
        departure TIME,
        duration VARCHAR(255),
        arrival TIME,
        rating FLOAT,
        price DECIMAL(10,2),
        availability INT
    )
""")

# Insert data into the table
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO bus_routes (name, bustype, departure, duration, arrival, rating, price, availability)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))

connection.commit()
cursor.close()
connection.close()


Streamlit Application

In [ ]:
import streamlit as st
import mysql.connector
import pandas as pd

# Connect to the database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="redbus_data"
)

# Query data
query = "SELECT * FROM bus_routes"
df = pd.read_sql(query, connection)

# Streamlit App
st.title("Redbus Data Filtering App")

# Filters
bus_type = st.multiselect("Select Bus Type", options=df["bustype"].unique())
price_range = st.slider("Select Price Range", min_value=0, max_value=int(df["price"].max()), value=(0, 500))

filtered_df = df[
    (df["bustype"].isin(bus_type)) & 
    (df["price"] >= price_range[0]) & 
    (df["price"] <= price_range[1])
]

st.dataframe(filtered_df)

connection.close()
